<a href="https://colab.research.google.com/github/sh-0620/dacon-recommender-system/blob/main/recommendations_TfidfVectorizer(all).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# CSV 파일 로드
log_df = pd.read_csv("/content/drive/MyDrive/웹 기사 추천시스템/view_log.csv")  # 유저 로그 데이터
articles_df = pd.read_csv("/content/drive/MyDrive/웹 기사 추천시스템/article_info.csv")  # 기사 데이터

In [ ]:
articles_df = articles_df.drop(['userCountry', 'userRegion', 'userID'], axis=1)

In [ ]:
merged_df = pd.merge(log_df, articles_df, on='articleID')

In [ ]:
# 기사 데이터 중 로그 데이터에 없는 기사 제거
articles_df  = articles_df[articles_df['articleID'].isin(merged_df['articleID'].unique())]

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42717 entries, 0 to 42716
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   userID       42717 non-null  object
 1   articleID    42717 non-null  object
 2   userRegion   42717 non-null  object
 3   userCountry  42717 non-null  object
 4   Title        42717 non-null  object
 5   Content      42717 non-null  object
 6   Format       42717 non-null  object
 7   Language     42717 non-null  object
dtypes: object(8)
memory usage: 2.6+ MB


In [ ]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2879 entries, 0 to 3007
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   articleID  2879 non-null   object
 1   Title      2879 non-null   object
 2   Content    2879 non-null   object
 3   Format     2879 non-null   object
 4   Language   2879 non-null   object
dtypes: object(5)
memory usage: 135.0+ KB


In [ ]:
log_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42717 entries, 0 to 42716
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   userID       42717 non-null  object
 1   articleID    42717 non-null  object
 2   userRegion   42717 non-null  object
 3   userCountry  42717 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


In [ ]:
# 라벨 인코더 생성 및 article_id 변환
# article_label_encoder = LabelEncoder()
# log_df['articleID'] = article_label_encoder.fit_transform(log_df['articleID'])

# 유저 아이디도 라벨 인코딩 적용
# user_label_encoder = LabelEncoder()
# log_df['userID'] = user_label_encoder.fit_transform(log_df['userID'])

In [ ]:
# 라벨 인코더 생성 및 학습
article_label_encoder = LabelEncoder()
merged_df['articleID'] = article_label_encoder.fit_transform(merged_df['articleID'])
articles_df['articleID'] = article_label_encoder.transform(articles_df['articleID'])

In [ ]:
# 유저 아이디 라벨 인코딩 적용
user_label_encoder = LabelEncoder()
merged_df['userID'] = user_label_encoder.fit_transform(merged_df['userID'])

In [ ]:
# log_df 원핫 인코딩
encoder = OneHotEncoder()

# 범주형 변수 선택 (지역, 나라)
log_df_categories = log_df[['userRegion', 'userCountry']]

# One-Hot Encoding 수행
log_df_encoded_categories = encoder.fit_transform(log_df_categories).toarray()

In [ ]:
# articles_df 원핫 인코딩
encoder = OneHotEncoder()

# 범주형 변수 선택 (언어, 형식)
articles_df_categories = articles_df[['Format', 'Language']]

# One-Hot Encoding 수행
articles_df_encoded_categories = encoder.fit_transform(articles_df_categories).toarray()

In [ ]:
# 기사 데이터 전처리: 제목 + 본문
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(articles_df['Title'] + ' ' + articles_df['Content'])

In [ ]:
# TF-IDF 행렬, articles_df 원핫 인코딩 / log_df 원핫 인코딩 / => 피처 벡터 결합
combined_features = np.hstack((tfidf_matrix.toarray(), articles_df_encoded_categories))


In [ ]:
# 유저별 기사 읽기 패턴 추출
user_read_articles = defaultdict(list)
for _, row in merged_df.iterrows():
    user_read_articles[row['userID']].append(row['articleID'])

In [ ]:
# 유저 기반 협업 필터링 모델
def recommend_articles(userID, top_n=5):
    if userID not in user_read_articles or not user_read_articles[userID]:

        return articles_df.sample(n=top_n)[['Title', 'Content']]


    read_articles = user_read_articles[userID]
    read_articles_features = combined_features[read_articles]

    if read_articles_features.shape[0] == 0:
        return articles_df.sample(n=top_n)[['Title', 'Content']]

    # 사용자가 읽은 기사들의 피처 벡터 평균 계산
    avg_features = np.mean(read_articles_features, axis=0).reshape(1, -1)

    # 사용자가 읽은 기사들의 피처 벡터 평균과 모든 기사들의 피처 벡터 간의 코사인 유사도 계산
    similarities = cosine_similarity(avg_features, combined_features)

    # 유사도가 높은 상위 top_n 개의 기사 인덱스 추출
    similar_articles_indices = similarities.flatten().argsort()[-top_n:][::-1]

    # 추천 기사 반환 (유저가 읽은 기사 포함)
    recommendations = articles_df.iloc[similar_articles_indices]
    return recommendations[['Title', 'Content']]

In [ ]:
user_recommendations = []

for encoded_user_id in merged_df['userID'].unique():
    # 추천 기사 인덱스 가져오기
    recommended_articles = recommend_articles(encoded_user_id)

    # 원래 유저 아이디로 역변환
    original_user_id = user_label_encoder.inverse_transform([encoded_user_id])[0]

    for idx in recommended_articles.index:
        # 원래 기사 아이디로 역변환
        original_article_id = article_label_encoder.inverse_transform([articles_df.loc[idx, 'articleID']])[0]

        # 추천 결과를 리스트에 저장
        user_recommendations.append({
            'userID': original_user_id,
            'articleID': original_article_id
        })

In [ ]:
print(user_recommendations)

[{'userID': 'USER_0000', 'articleID': 'ARTICLE_2255'}, {'userID': 'USER_0000', 'articleID': 'ARTICLE_0411'}, {'userID': 'USER_0000', 'articleID': 'ARTICLE_1270'}, {'userID': 'USER_0000', 'articleID': 'ARTICLE_2081'}, {'userID': 'USER_0000', 'articleID': 'ARTICLE_1305'}, {'userID': 'USER_0067', 'articleID': 'ARTICLE_2991'}, {'userID': 'USER_0067', 'articleID': 'ARTICLE_1891'}, {'userID': 'USER_0067', 'articleID': 'ARTICLE_1119'}, {'userID': 'USER_0067', 'articleID': 'ARTICLE_0120'}, {'userID': 'USER_0067', 'articleID': 'ARTICLE_2721'}, {'userID': 'USER_0414', 'articleID': 'ARTICLE_0520'}, {'userID': 'USER_0414', 'articleID': 'ARTICLE_0220'}, {'userID': 'USER_0414', 'articleID': 'ARTICLE_1385'}, {'userID': 'USER_0414', 'articleID': 'ARTICLE_2530'}, {'userID': 'USER_0414', 'articleID': 'ARTICLE_1891'}, {'userID': 'USER_0557', 'articleID': 'ARTICLE_2858'}, {'userID': 'USER_0557', 'articleID': 'ARTICLE_2422'}, {'userID': 'USER_0557', 'articleID': 'ARTICLE_1449'}, {'userID': 'USER_0557', 'ar

In [ ]:
recommendations_df = pd.DataFrame(user_recommendations)
recommendations_df.to_csv('user_recommendations.csv', index=False)